<a href="https://colab.research.google.com/github/cundeyu154/PL-Repo/blob/main/HW1%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E6%95%B8%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

日常支出速算與分攤（作業一）


目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。

AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。

Sheet 欄位：date, category, item, amount, payer


Google Sheet:https://docs.google.com/spreadsheets/d/1unp8V4uw_zv31d3x9GTWY2Qh5TKvMWYdcpSjyKJar0Q/edit?gid=0#gid=0

In [ ]:
# 安裝必要的函式庫
# gspread 和 pandas 已經安裝過，這裡保留以防萬一
!pip install gspread pandas

#  匯入函式庫
import gspread
import pandas as pd
from google.colab import auth

print("函式庫安裝與匯入完成！")

print("\n請點擊連結完成 Google 授權：")
auth.authenticate_user()
print("Google 授權完成！")
# 使用 google.auth 的憑證來授權 gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
print("gspread 授權成功。")


spreadsheet_name = "HW1日常支出數算與分攤"
worksheet_name = "Sheet1"

try:
    sh = gc.open(spreadsheet_name)
    worksheet = sh.worksheet(worksheet_name)

    try:
        summary_ws = sh.worksheet("Summary")
        print("Summary 分頁已存在，直接使用。")
    except gspread.WorksheetNotFound:
        summary_ws = sh.add_worksheet(title="Summary", rows="100", cols="20")
        print("Summary 分頁已創建並準備好。")

    print("試算表連接成功。")
except gspread.WorksheetNotFound:
    print(f"錯誤：找不到資料分頁: {worksheet_name}。請檢查分頁名稱。")
    # 如果找不到，這裡讓程式停止執行
    raise
except gspread.SpreadsheetNotFound:
    print(f"錯誤：找不到試算表: {spreadsheet_name}。請檢查檔案名稱。")
    # 如果找不到，這裡讓程式停止執行
    raise

函式庫安裝與匯入完成！

請點擊連結完成 Google 授權：
Google 授權完成！
gspread 授權成功。
Summary 分頁已存在，直接使用。
試算表連接成功。


In [ ]:
import datetime

# --- 1. 互動式詢問使用者輸入 ---
print("--- 請輸入新增的消費紀錄 ---")

# 詢問日期並檢查格式
while True:
    date_str = input("請輸入日期 (格式：YYYY-MM-DD): ")
    try:
        datetime.datetime.strptime(date_str, '%Y-%m-%d')
        break
    except ValueError:
        print("日期格式錯誤，請重新輸入 (YYYY-MM-DD)。")

# 詢問時間並檢查格式
while True:
    time_str = input("請輸入時間 (格式：HH:MM): ")
    try:
        datetime.datetime.strptime(time_str, '%H:%M')
        break
    except ValueError:
        print("時間格式錯誤，請重新輸入 (HH:MM)。")

# 詢問品項和金額
item = input("請輸入品項: ")
# 確保金額是數字
while True:
    try:
        amount_str = input("請輸入金額: ")
        amount = float(amount_str)
        break
    except ValueError:
        print("金額必須是數字，請重新輸入。")

# 詢問分類和付款人 (假設您的資料表中有這兩個欄位)
# 註：如果您的資料表沒有 '分類' 或 '付款人' 欄位，請將下面兩行刪除。
category = input("請輸入分類 (例如：餐飲外食): ")
payer = input("請輸入付款人名稱: ")

print(f"\n您輸入的資料為: {date_str}, {time_str}, {item}, {amount}, {category}, {payer}")

# --- 2. 準備寫入資料 (list of lists 格式) ---

new_row_data = [
    [date_str, time_str, item, amount, category, payer]
]


# --- 3. 寫入 Google Sheets ---

try:
    # 'worksheet' 是您在第一個儲存格中定義的 'Sheet1' 或其他名稱的工作表對象
    worksheet.append_rows(values=new_row_data, value_input_option='USER_ENTERED')
    print("\n✅ 資料已成功新增到 Google Sheets 的工作表！")

except NameError:
    print("\n❌ 錯誤：請先執行第一個儲存格以完成 gspread 授權和工作表連接。")

--- 請輸入新增的消費紀錄 ---
請輸入日期 (格式：YYYY-MM-DD): 2025-09-19
請輸入時間 (格式：HH:MM): 06:00
請輸入品項: 牛奶
請輸入金額: 60
請輸入分類 (例如：餐飲外食): 飲料
請輸入付款人名稱: B

您輸入的資料為: 2025-09-19, 06:00, 牛奶, 60.0, 飲料, B

✅ 資料已成功新增到 Google Sheets 的工作表！


In [ ]:
# 讀取資料並轉換為 DataFrame
data = worksheet.get_all_records()
df = pd.DataFrame(data)

# 關鍵修正：使用中文欄位名稱 '金額' 進行數字轉換
df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
df.dropna(subset=['金額'], inplace=True)

print("數據讀取成功，開始計算...")

# --- 總額計算 ---
total_amount = df['金額'].sum()
total_records = len(df)
print(f"\n總消費額: {total_amount:.2f}")

# --- 分類小計 (Category Subtotals) ---
category_subtotals = df.groupby('分類')['金額'].sum().reset_index()
category_subtotals.columns = ['Category', 'Subtotal']

# --- AA 分攤計算 (Split Payments) ---
all_payers = df['付款人'].unique()
num_payers = len(all_payers)
average_share = total_amount / num_payers

# 計算每人實際支付總額
payer_payments = df.groupby('付款人')['金額'].sum().reindex(all_payers, fill_value=0).reset_index()
payer_payments.columns = ['Payer', 'Paid_Total']

# 計算每人應收/應付 (Balance)
payer_payments['Share_Amount'] = average_share
payer_payments['Balance'] = payer_payments['Paid_Total'] - payer_payments['Share_Amount']

print(f"\nAA 分攤計算完成 (每人平均應分攤: {average_share:.2f})")
print(payer_payments[['Payer', 'Paid_Total', 'Balance']])

數據讀取成功，開始計算...

總消費額: 835.00

AA 分攤計算完成 (每人平均應分攤: 417.50)
  Payer  Paid_Total  Balance
0     A         410     -7.5
1     B         425      7.5


In [ ]:
# 清空 Summary 分頁 (確保結果是最新的)
summary_ws.clear()

# 寫入總額
summary_ws.update_cell(1, 1, "總結報告")
summary_ws.update_cell(2, 1, "總消費額")
# 修正：轉換為標準 float 類型
summary_ws.update_cell(2, 2, float(total_amount))
summary_ws.update_cell(3, 1, "總筆數")
summary_ws.update_cell(3, 2, total_records)

print("\n基礎資訊寫入 Summary 分頁...")


# 寫入分類小計 (從 A5 開始)
summary_ws.update_cell(5, 1, "分類小計")
# 將 DataFrame 轉換為列表
subtotals_list = [category_subtotals.columns.tolist()] + category_subtotals.values.tolist()
# ★★★ 關鍵修正：將 update_cells 替換為 update ★★★
# 計算寫入範圍，例如 A6:B8
subtotals_range = f'A6:B{6 + len(subtotals_list) - 1}'
summary_ws.update(range_name=subtotals_range, values=subtotals_list)


# 寫入 AA 分攤結果 (從 D5 開始)
summary_ws.update_cell(5, 4, "AA 分攤結果")
summary_ws.update_cell(6, 4, f"每人平均應分攤: {average_share:.2f}")

# 只取出 Payer, Paid_Total 和 Balance 進行寫入
aa_output = payer_payments[['Payer', 'Paid_Total', 'Balance']]
aa_list = [aa_output.columns.tolist()] + aa_output.values.tolist()

# ★★★ 關鍵修正：將 update_cells 替換為 update ★★★
# 計算寫入範圍，例如 D8:F9
aa_range = f'D8:F{8 + len(aa_list) - 1}'
summary_ws.update(range_name=aa_range, values=aa_list)


print("\n結果已成功寫入 Google Sheets 的 Summary 分頁！")

#  組織數據給 AI 分析
analysis_data = f"""

總消費額: {total_amount:.2f}
分類小計:
"""

# 將分類小計結果加入
for index, row in category_subtotals.iterrows():
    analysis_data += f"* {row['Category']}: {row['Subtotal']:.2f}\n"

print(analysis_data)


基礎資訊寫入 Summary 分頁...

結果已成功寫入 Google Sheets 的 Summary 分頁！


總消費額: 835.00
分類小計:
* cake: 70.00
* 交通: 40.00
* 飲料: 195.00
* 餐飲外食: 530.00



In [ ]:
!pip install gspread pandas gradio
import gspread
import pandas as pd
from google.colab import auth
import datetime
import gradio as gr
import os
import numpy as np

print("✅ 函式庫安裝與匯入完成！")

spreadsheet_name = "HW1日常支出數算與分攤"
worksheet_name = "Sheet1"
gc = None

✅ 函式庫安裝與匯入完成！


In [ ]:
def authenticate_and_get_client():
    """處理 Google 授權並取得 gspread 客戶端。"""
    global gc
    if gc is None:
        try:
            print("\n請點擊連結完成 Google 授權：")
            auth.authenticate_user()
            print("Google 授權完成！")
            from google.auth import default
            creds, _ = default()
            gc = gspread.authorize(creds)
            print("gspread 授權成功。")
            return gc
        except Exception as e:
            print(f"❌ 授權失敗: {e}")
            return None
    else:
        return gc
gc = authenticate_and_get_client()


請點擊連結完成 Google 授權：
Google 授權完成！
gspread 授權成功。


In [ ]:
def run_analysis(gc):
    """
    從 Google Sheets 讀取資料，執行計算，並回傳結果 DataFrame 和分析文字。
    """
    if gc is None:
        return pd.DataFrame(), pd.DataFrame(), "❌ 錯誤：gspread 尚未授權。請先執行步驟二完成授權。"

    try:
        sh = gc.open(spreadsheet_name)
        worksheet = sh.worksheet(worksheet_name)
        try:
            summary_ws = sh.worksheet("Summary")
        except gspread.WorksheetNotFound:
            summary_ws = sh.add_worksheet(title="Summary", rows="100", cols="20")

        print(f"試算表 '{spreadsheet_name}' 連接成功，開始讀取數據...")
        data = worksheet.get_all_records()
        df = pd.DataFrame(data)

        if df.empty:
            return pd.DataFrame(), pd.DataFrame(), "⚠️ 數據分頁為空，無法進行分析。"

        df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
        df.dropna(subset=['金額'], inplace=True)

        if df.empty:
            return pd.DataFrame(), pd.DataFrame(), "⚠️ 數據分頁中 '金額' 欄位無有效數字，無法進行分析。"
        total_amount = df['金額'].sum()
        total_records = len(df)

        category_subtotals = df.groupby('分類')['金額'].sum().reset_index()
        category_subtotals.columns = ['分類', '金額小計']

        all_payers = df['付款人'].unique()
        num_payers = len(all_payers)
        average_share = total_amount / num_payers if num_payers > 0 else 0

        payer_payments = df.groupby('付款人')['金額'].sum().reindex(all_payers, fill_value=0).reset_index()
        payer_payments.columns = ['付款人', '已付總額']
        payer_payments['應分攤金額'] = average_share
        payer_payments['淨額'] = payer_payments['已付總額'] - payer_payments['應分攤金額']

        payer_payments_gradio = payer_payments.copy()
        payer_payments_gradio[['已付總額', '應分攤金額', '淨額']] = payer_payments_gradio[['已付總額', '應分攤金額', '淨額']].applymap('{:,.2f}'.format)
        summary_ws.clear()
        summary_ws.update_cell(1, 1, "總結報告")
        summary_ws.update_cell(2, 1, "總消費額")
        summary_ws.update_cell(2, 2, float(total_amount))
        summary_ws.update_cell(3, 1, "總筆數")
        summary_ws.update_cell(3, 2, total_records)

        summary_ws.update_cell(4, 1, "每人平均應分攤")
        summary_ws.update_cell(5, 1, "分類小計")
        subtotals_list = [category_subtotals.columns.tolist()] + category_subtotals.values.tolist()
        summary_ws.update(range_name=f'A6:B{6 + len(subtotals_list) - 1}', values=subtotals_list)

        summary_ws.update_cell(5, 4, "AA 分攤結果")
        aa_list = [payer_payments.columns.tolist()] + payer_payments.values.tolist()
        aa_output_ws = payer_payments.copy()
        aa_list = [aa_output_ws.columns.tolist()] + aa_output_ws.values.tolist()
        data_rows = len(aa_list)
        aa_range = f'D8:G{8 + data_rows - 1}'


        summary_ws.update(range_name=aa_range, values=aa_list)

        print("✅ 結果已成功寫入 Google Sheets 的 Summary 分頁！")


        analysis_text = f"💡 **日常支出分析報告** 💡\n\n"
        analysis_text += f"**總消費額**: NT${total_amount:,.2f} (共 {total_records} 筆紀錄)\n"
        analysis_text += f"**每人平均應分攤**: NT${average_share:,.2f} (共 {num_payers} 人)\n"

        analysis_text += "\n--- **分類高光** ---\n"
        if not category_subtotals.empty:
            max_cat = category_subtotals.iloc[category_subtotals['金額小計'].argmax()]
            min_cat = category_subtotals.iloc[category_subtotals['金額小計'].argmin()]

            analysis_text += f"💸 **最大支出分類**: **{max_cat['分類']}**，共 NT${max_cat['金額小計']:,.2f}。\n"
            analysis_text += f"🌱 **最小支出分類**: **{min_cat['分類']}**，共 NT${min_cat['金額小計']:,.2f}。\n"
            analysis_text += f"\n👉 **建議**: 檢視 '{max_cat['分類']}' 是否有節省空間。\n"

        analysis_text += "\n--- **AA 分攤淨額摘要** ---\n"

        should_receive = payer_payments[payer_payments['淨額'] > 0].sort_values(by='淨額', ascending=False)
        should_pay = payer_payments[payer_payments['淨額'] < 0].sort_values(by='淨額', ascending=True)

        if not should_receive.empty:
            receiver = should_receive.iloc[0]
            analysis_text += f"💰 **最高應收款**: **{receiver['付款人']}**，應收 **{receiver['淨額']:,.2f}** 元。\n"

        if not should_pay.empty:
            payer = should_pay.iloc[0]
            analysis_text += f"📉 **最高應付款**: **{payer['付款人']}**，應付 **{-payer['淨額']:,.2f}** 元。\n"

        analysis_text += "\n**請參考下方的 AA 分攤結果表格，進行結算。**"

        return category_subtotals, payer_payments_gradio[['付款人', '已付總額', '應分攤金額', '淨額']], analysis_text

    except gspread.WorksheetNotFound:
        return pd.DataFrame(), pd.DataFrame(), f"❌ 錯誤：找不到資料分頁: {worksheet_name}。請檢查分頁名稱。"
    except gspread.SpreadsheetNotFound:
        return pd.DataFrame(), pd.DataFrame(), f"❌ 錯誤：找不到試算表: {spreadsheet_name}。請檢查檔案名稱。"
    except Exception as e:
        return pd.DataFrame(), pd.DataFrame(), f"❌ 發生未知錯誤: {e}"

def gradio_wrapper():
    """確保每次都使用全域的 gc 客戶端來呼叫分析函數。"""
    return run_analysis(gc)

In [ ]:
def gradio_wrapper():
    """確保每次都使用全域的 gc 客戶端來呼叫分析函數。"""
    global gc
    return run_analysis(gc)

with gr.Blocks(title="日常支出數算與分攤報告") as demo:
    gr.Markdown("# 💸 Google Sheets 日常支出數算與分攤報告 📊")
    ai_analysis_output = gr.Markdown(label="AI 分析結果", value="點擊下方按鈕開始分析...")
    run_button = gr.Button("點擊我更新並執行分析")

    gr.Markdown("---")
    gr.Markdown("## 📋 計算詳細結果")

    with gr.Row():
        category_df_output = gr.DataFrame(
            headers=["分類", "金額小計"],
            label="分類支出小計"
        )
        aa_df_output = gr.DataFrame(
            headers=["付款人", "已付總額", "應分攤金額", "淨額"],
            label="AA 分攤淨額 (+號為應收, -號為應付)"
        )

    # 5. 定義互動邏輯 (按鈕點擊觸發分析)
    run_button.click(
        fn=gradio_wrapper,
        inputs=None,
        outputs=[category_df_output, aa_df_output, ai_analysis_output]
    )

    # 6. 在頁面加載時自動執行一次分析
    demo.load(
        fn=gradio_wrapper,
        inputs=None,
        outputs=[category_df_output, aa_df_output, ai_analysis_output]
    )

# 7. 啟動 Gradio 介面
print("\n--- 啟動 Gradio 介面 ---")
demo.launch(debug=True, share=True)


--- 啟動 Gradio 介面 ---
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b1e43bca951e184a20.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


試算表 'HW1日常支出數算與分攤' 連接成功，開始讀取數據...


/tmp/ipython-input-3568861993.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  payer_payments_gradio[['已付總額', '應分攤金額', '淨額']] = payer_payments_gradio[['已付總額', '應分攤金額', '淨額']].applymap('{:,.2f}'.format)


✅ 結果已成功寫入 Google Sheets 的 Summary 分頁！
試算表 'HW1日常支出數算與分攤' 連接成功，開始讀取數據...


/tmp/ipython-input-3568861993.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  payer_payments_gradio[['已付總額', '應分攤金額', '淨額']] = payer_payments_gradio[['已付總額', '應分攤金額', '淨額']].applymap('{:,.2f}'.format)


...

✅ 結果已成功寫入 Google Sheets 的 Summary 分頁！
試算表 'HW1日常支出數算與分攤' 連接成功，開始讀取數據...


/tmp/ipython-input-3568861993.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  payer_payments_gradio[['已付總額', '應分攤金額', '淨額']] = payer_payments_gradio[['已付總額', '應分攤金額', '淨額']].applymap('{:,.2f}'.format)


✅ 結果已成功寫入 Google Sheets 的 Summary 分頁！
試算表 'HW1日常支出數算與分攤' 連接成功，開始讀取數據...


/tmp/ipython-input-3568861993.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  payer_payments_gradio[['已付總額', '應分攤金額', '淨額']] = payer_payments_gradio[['已付總額', '應分攤金額', '淨額']].applymap('{:,.2f}'.format)


✅ 結果已成功寫入 Google Sheets 的 Summary 分頁！
